In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Simple perceptron w/sklearn API + Iris classification

In [4]:
iris = load_iris()
X = iris.data[:, (2,3)] # only petal length and width
y = (iris.target == 0).astype(int)  # make binary "is it a setosa?"

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

Perceptron()

In [6]:
print(perceptron.score(X_test, y_test)) # given petal length of 2 and width of 0.5, is it a setosa?

1.0


## Simple perceptron from scratch w/Iris classification

A single neuron/node with:
- 2 inputs (petal length and width) and 2 input weights
- 1 bias input (always inputs 1) and 1 bias weight
- 1 binary output (1 == "is setosa", 0 == "is not setosa")

Learning process:
- For each input instance in training set:
    - multiply each input feature (petal length and width) by its weight, and multiply the bias by its weight
    - Sum the results
    - If greater than or equal to 0, output 1, else output 0
    - Calculate error by minusing the output from the target output. 
    - For each weight:
        - Add the ((error * learning rate) * input) to the weight
        - e.g. if the weight was 0.5, the input was -0.7, the output was 1 and the target output was 0, that's an error of -1, 0.05 * -1 * -0.7 = 0.035, so new weight is 0.535. If you got the same input again you've amplified the negative input which will result in a sum of weights * inputs closer to being less than 0, resulting in an output of 0, the target.
        - e.g. if the weight was 0.5, the input was 0.7, the output 1 and the target output was 0, that's error of -1, 0.05 * -1 * 0.7 = -0.035, so the new weight is 0.465. If you got the same input again, you've supressed the positive input which will result in a sum of weights * inputs closer to being less than 0, resulting in an output of 0, the target.
       


In [7]:
class MyPerceptron:
    def __init__(self):   
        self.input_weights = [0.5, 0.5]
        self.bias_weight = 0.5
        self.learning_rate = 0.05
    
    def fit(self, X, y):
        for i in range(0, X.shape[0]):
            output = self.predict(X[i, :])
            error = y[i] - output
            for j in range(0, len(self.input_weights)):
                self.input_weights[j] = self.input_weights[j] + ((self.learning_rate * error) * X[i, j])
            self.bias_weight = self.bias_weight + ((self.learning_rate * error) * 1)
            
    def predict(self, x):
        product = 0
        for input_val, weight in zip(x, self.input_weights):
            product += input_val * weight
        product += 1 * self.bias_weight
        return int(product >= 0) # simple heaviside step function
    
    def score(self, X, y):
        error = y.shape[0]
        for i in range(0, X.shape[0]):
            output = self.predict(X[i, :])
            error -= abs(y[i] - output)
        return f"{error}/{y.shape[0]}"

In [8]:
model = MyPerceptron()
print(model.input_weights)
print(model.score(X_test,y_test))
model.fit(X_train, y_train)
print(model.input_weights)
print(model.score(X_test,y_test))

[0.5, 0.5]
11/38
[-0.19, 0.245]
38/38


## Simple perceptron continous output using backpropagation

In [9]:
import math

def sigmoid(z):
    return 1 / (1 + math.exp(-z))

def half_mse(a, y):
    return 0.5 * ((a - y)**2)   # half MSE is nicer derivative than MSE


class BackPropagatingPerceptron:
    def __init__(self):
        self.a_out = []   # output after sigmoid - is a matrix because storing output from multiple input instances
        #self.z_out = []   # doesn't need storing for backprop
        self.w_out = [0.5, 0.5]
        self.b_out = 0.5
        
        self.l_rate = 0.5
    
    def forward_pass(self, X, y):
        cost_out = []
        for i in range(0, X.shape[0]):
            z_out = 0
            for j in range(0, len(self.w_out)):
                z_out += self.w_out[j] * X[i, j]
            z_out += self.b_out
            squished = sigmoid(z_out)
            self.a_out.append(squished) # matrix cus for hidden layers it'd be multiple outputs per input instance
            cost = half_mse(squished, y[i])
            cost_out.append(cost)
        return sum(cost_out)
    
    def backwards_pass(self, X, y):
        w_delta_out = [[],[]] # 1 gradient per weight per input instance
        b_delta_out = [] # 1 gradient per input instance
        for i in range(0, X.shape[0]): # i = instance
            for j in range(0, len(self.w_out)): # j = weight
                print(-(y[i] - self.a_out[i]))
                w_delta_out[j].append((-(y[i] - self.a_out[i])) * (self.a_out[i]*(1-self.a_out[i])) * X[i, j])  
            b_delta_out.append((-(y[i] - self.a_out[i])) * (self.a_out[i]*(1-self.a_out[i])) * 1)
        
        for i in range(0, len(w_delta_out)):
            self.w_out[i] = self.w_out[i] - (self.l_rate * (sum(w_delta_out[i]) / len(w_delta_out[i])))
        self.b_out = self.b_out - (self.l_rate * (sum(b_delta_out) / len(b_delta_out)))
        
        # why are my gradient exploding after a while? read le book

In [10]:
diabetes = load_diabetes()
X = diabetes.data[:, (0,3)]
y = diabetes.target

scaler = MinMaxScaler()  # because output is sigmoid
y = scaler.fit_transform(y.reshape(-1, 1))
y = y.reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
model = BackPropagatingPerceptron()
for epoch in range(0, 1):
    if epoch % 10 == 0:
        print(model.forward_pass(X_train, y_train))
    model.forward_pass(X_train, y_train)
    model.backwards_pass(X_train, y_train)

16.267954843453833
0.492323826017706
0.492323826017706
-0.33377166235073197
-0.33377166235073197
0.44430822385815844
0.44430822385815844
0.09854586916946284
0.09854586916946284
0.2825804908397682
0.2825804908397682
-0.10397523191211089
-0.10397523191211089
0.12432666890417848
0.12432666890417848
0.19641407946463096
0.19641407946463096
0.2867529939207341
0.2867529939207341
0.3863509675118189
0.3863509675118189
0.25409536840936553
0.25409536840936553
-0.12901138215201757
-0.12901138215201757
0.3254532731312581
0.3254532731312581
0.1786937112585531
0.1786937112585531
0.025153633588736457
0.025153633588736457
0.11156875910579
0.11156875910579
0.3211308151987371
0.3211308151987371
0.1673342559775689
0.1673342559775689
0.26314093901363717
0.26314093901363717
0.266013110480909
0.266013110480909
0.2859761400293867
0.2859761400293867
0.2224581025020831
0.2224581025020831
0.15685732421342813
0.15685732421342813
0.34826412514281074
0.34826412514281074
0.2916147706530802
0.2916147706530802
0.17183

In [46]:
import random

class MultiLayerPerceptron:
    def __init__(self):
        self.layer_h1 = Layer(n_nodes = 1, n_inputs = 2)
        self.layer_out = Layer(n_nodes = 1, n_inputs = 1)
    
    def forward_pass(self, X, y):
        out_h1 = self.layer_h1.forward_pass(X)
        out_out = self.layer_out.forward_pass(out_h1)
        cost_out = []
        for i in range(len(y)):
            cost = half_mse(out_out[0][i], y[i])
            cost_out.append(cost)
        return sum(cost_out)
    
    def backwards_pass(self, X, y):
        cost_delta = []
        for i in range(X.shape[0]):
            cost_delta.append(-(y[i] - self.layer_out.nodes[0].outputs[i]))
        z_delta_next = [[cost_delta]] # [prev_node_1[curr_node1_deltas[instances], curr_node2_delta[instance]], prev_node_2...]
            
        self.layer_out.backwards_pass(X, z_delta_next)
            

class Layer:
    def __init__(self, n_nodes, n_inputs):
        self.nodes = []
        for i in range(n_nodes):
            self.nodes.append(Node(n_inputs))
    
    def forward_pass(self, X):
        outputs = []
        for i in range(len(self.nodes)):
            outputs.append(self.nodes[i].forward_pass(X))
        return outputs
    
    def backwards_pass(self, X, z_delta_next):
        for i in range(len(self.nodes)):
            self.nodes[i].backwards_pass(X, z_delta_next[i])
        
class Node:
    def __init__(self, n_inputs):
        self.outputs = []
        self.weights = []
        self.bias = random.random()
        self.l_rate = 0.5
         
        for i in range(n_inputs):
            self.weights.append(random.random())
    
    def forward_pass(self, X):
        self.outputs = []
        out = []
        for i in range(len(X[0])):
            z_sum = 0
            for j in range(len(self.weights)):
                z_sum += self.weights[j] * X[j][i]
            z_sum += self.bias
            self.outputs.append(sigmoid(z_sum))
        return self.outputs
    
    def backwards_pass(self, X, z_delta_next):
        w_delta_matrix = [[] for i in range(len(self.weights))]
        b_delta_array = []
        for i in range(0, X.shape[0]):
            z_delta = self.outputs[i]*(1-self.outputs[i])
            for j in range(len(self.weights)):
                part_w_delta = z_delta * X[i, j]
                full_w_delta = 0
                for k in range(len(z_delta_next)):
                    full_w_delta += z_delta_next[k][i] * part_w_delta
                w_delta_matrix[j].append(full_w_delta)
            b_delta_array.append(self.outputs[i]*(1-self.outputs[i]))
        
        for i in range(len(w_delta_matrix)):
            self.weights[i] = self.weights[i] - (self.l_rate * (sum(w_delta_matrix[i]) / len(w_delta_matrix[i])))
        self.bias = self.bias - (self.l_rate * (sum(b_delta_array) / len(b_delta_array)))
        
        
# return z_delta * the weights

# once I sort the input into backwards_pass() and output of backward_pass()... it should work with any number of
# layers and nodes?

# the thing that's making this confusing is having to keep track of the instances in the main backprop code...
# could that loop be moved elsewhere somehow?

# I need to add to a z_delta_next for each node in prevous layer during backprop
# For each node in previous layer, each node should add its z_delta multiplied by the weight for the prev layer node

In [47]:
model = MultiLayerPerceptron()
for epoch in range(0, 1):
    print(model.forward_pass(X_train.reshape(2, 331), y_train))
    #model.backwards_pass(X_train, y_train)
    #print(model.layer_out.nodes[0].weights)

24.10634585254685


In [26]:
X_train.shape

(331, 2)

In [27]:
X_train.reshape(2, 331)

array([[ 1.56036774,  1.12118221, -2.07571417, -1.557304  ,  1.24418671,
         0.98020926, -1.60144262, -0.71146624, -1.8385784 , -0.50000681,
         0.92800567,  0.13437151, -0.17862796, -0.71146624,  0.69086989,
        -0.35903385,  0.37468886, -0.50000681,  0.05850782, -0.28854737,
        -0.65289951,  1.40312813,  0.76991515,  1.54410109, -0.57385425,
        -0.07708793, -1.68048788, -0.28854737,  0.2956436 , -1.18161106,
         0.37468886, -0.07708793, -1.8385784 , -0.00660145, -0.57385425,
         0.98020926, -1.12717107, -1.41633104,  0.05850782, -0.71146624,
         0.21659834, -1.13438512, -1.52239736, -0.8524392 ,  0.21659834,
         0.55729038,  1.48132248, -1.76876343,  0.21659834,  0.41631742,
        -0.41576373, -2.05070935,  1.48132248,  0.06388503, -0.65289951,
         0.27534446,  0.84896041,  0.01665909,  0.76991515, -0.28854737,
        -0.49480899, -0.28854737, -0.41576373,  1.82604701, -0.73194477,
        -0.38229439, -1.04812581, -0.02986199,  0.1